In [1]:
import pandas as pd
import os
from sksurgerynditracker.nditracker import NDITracker
import numpy as np

In [2]:
TOOL_TIP_OFFSET = np.array([-304.5728,-0.3053,-0.1412, 1]) # the offset value is from Mike github repo
# Initialize the NDI tracker
SETTINGS = {
    "tracker type": "vega",
    "ip address": "169.254.7.250",
    "port": 8765,
    "romfiles": ["/Users/yizheng/Desktop/Dartmouth/lab work/ARPAH/Polaris Vega XT/tool_defs/medtronic_chicken_foot_960_556.rom"]
}
tracker = NDITracker(SETTINGS)
tracker.start_tracking()

PING 169.254.7.250 (169.254.7.250): 56 data bytes
64 bytes from 169.254.7.250: icmp_seq=0 ttl=64 time=0.541 ms

--- 169.254.7.250 ping statistics ---
1 packets transmitted, 1 packets received, 0.0% packet loss
round-trip min/avg/max/stddev = 0.541/0.541/0.541/nan ms


In [3]:
def get_tooltip_data(tool_tip_offset):
    port_handles, timestamps, framenumbers, tracking, quality = tracker.get_frame()
    tool_tip = np.dot(tracking[0], tool_tip_offset)
    x_pos = tool_tip[0]
    y_pos = tool_tip[1]
    z_pos = tool_tip[2]
    return x_pos, y_pos, z_pos

In [22]:
fcsv_file = 'CT.fcsv'
phantom_fiducials = pd.read_csv(os.getcwd() + '/' + 'slicer_files' + '/' + fcsv_file, comment='#', header=None)

In [7]:
real_pos = []
i = 0
while i < len(phantom_fiducials):
    user_input = input("Press Enter to Collect Points：")
        
    if user_input.lower() == 'Exit':
        break

    if user_input == '':  # 如果按回车键
        x, y, z = get_tooltip_data(TOOL_TIP_OFFSET)  # 获取当前Polaris的位置
        if np.isnan(x) or np.isnan(y) or np.isnan(z):
            print("Collect again")
        else:
            real_pos.append([x, y, z])  # 保存采集的数据       
            i = i + 1
            print(f"Collected Point: {i}")
            print(f"Position: {x, y, z}")
            
    

Collect again
Collect again
Collect again
Collected Point: 1
Position: (np.float64(40.51320564842001), np.float64(-253.87461016031196), np.float64(-1379.9160380047042))
Collected Point: 2
Position: (np.float64(-15.793435728028072), np.float64(-49.100365838854), np.float64(-1414.316930792986))
Collected Point: 3
Position: (np.float64(-59.60176576511191), np.float64(-159.266586397558), np.float64(-1465.129753680758))
Collected Point: 4
Position: (np.float64(-55.64992615989195), np.float64(-174.128708296206), np.float64(-1574.3612435942562))
Collected Point: 5
Position: (np.float64(0.5209652173360837), np.float64(-85.659662666812), np.float64(-1636.709391531904))
Collected Point: 6
Position: (np.float64(96.21857272648998), np.float64(-310.633344157286), np.float64(-1595.5782589503021))


In [43]:
from scipy.spatial.transform import Rotation
def compute_rigid_transform(P_phantom, P_polaris):
    """
    计算从 phantom 坐标系到 polaris 坐标系的刚体变换 (R, T)
    
    :param P_phantom: ndarray (N,3)，phantom 坐标系下 fiducials 的 3D 坐标
    :param P_polaris: ndarray (N,3)，Polaris 坐标系下 fiducials 的 3D 坐标
    :return: R (3x3 rotation matrix), T (3x1 translation vector)
    """
    # 计算两个点集的质心
    centroid_phantom = np.mean(P_phantom, axis=0)
    centroid_polaris = np.mean(P_polaris, axis=0)

    # 去中心化（中心化点集）
    P_phantom_centered = P_phantom - centroid_phantom
    P_polaris_centered = P_polaris - centroid_polaris

    # 计算协方差矩阵 H
    H = P_phantom_centered.T @ P_polaris_centered

    # 进行 SVD 分解
    U, _, Vt = np.linalg.svd(H)
    
    # 计算旋转矩阵 R
    R = Vt.T @ U.T

    # 处理可能的反射问题（保证 R 是正交矩阵，det(R) = 1）
    if np.linalg.det(R) < 0:
        Vt[-1, :] *= -1
        R = Vt.T @ U.T

    # 计算平移向量 T
    T = centroid_polaris - R @ centroid_phantom

    return R, T

In [45]:
P_phantom = []
phantom_fiducials = np.array(phantom_fiducials)
for i in range(len(phantom_fiducials)):
    P_phantom.append(phantom_fiducials[i][1:4])
P_phantom = np.array(P_phantom, dtype=np.float64)
print(P_phantom, )

[[-108.610672    -93.25263214  128.33157349]
 [  98.98032379 -147.69650269  129.06384277]
 [  -0.64749622 -195.0091095   193.43154907]
 [   1.81817102 -195.08876038  301.2645874 ]
 [  98.23664093 -144.80683899  351.45755005]
 [-134.05921936  -50.59602356  351.95401001]]


In [46]:
P_polaris = np.array(real_pos, dtype=np.float64)
print(P_polaris)

[[ 4.05132056e+01 -2.53874610e+02 -1.37991604e+03]
 [-1.57934357e+01 -4.91003658e+01 -1.41431693e+03]
 [-5.96017658e+01 -1.59266586e+02 -1.46512975e+03]
 [-5.56499262e+01 -1.74128708e+02 -1.57436124e+03]
 [ 5.20965217e-01 -8.56596627e+01 -1.63670939e+03]
 [ 9.62185727e+01 -3.10633344e+02 -1.59557826e+03]]


In [47]:
R, T = compute_rigid_transform(P_phantom, P_polaris)


In [50]:
def create_transformation_matrix(R, T):
    """
    将旋转矩阵 R (3x3) 和平移向量 T (3x1) 组合成 4x4 变换矩阵
    """
    transformation_matrix = np.eye(4)  # 创建一个 4x4 单位矩阵
    transformation_matrix[:3, :3] = R  # 设置旋转部分
    transformation_matrix[:3, 3] = T.flatten()  # 设置平移部分
    return transformation_matrix

In [51]:
transformation_matrix = create_transformation_matrix(R, T)
print(transformation_matrix)

[[-1.30866820e-02  9.98415042e-01  5.47370336e-02  1.25183027e+02]
 [ 9.88019474e-01  2.13301560e-02 -1.52848106e-01 -1.24802059e+02]
 [-1.53773398e-01  5.20809806e-02 -9.86732645e-01 -1.26559772e+03]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
